In [1]:
# TASK #1_DATA (for DATA group, submit solution via e-mail)

# Write a script to ingest the data from the file task1_d.json into a relational database (note that it's not a valid JSON, you need to process it).

# Apply transformations to produce a summary table with the following fields:
# publication year,
# book_count published that year,
# average_price of books published that year in USD rounded to cents using convertion rate €1 = $1.2.

# You can use any language for uploading data, but transformation is supposed to be inside the database.

# Record the video demonstrating all the code you've wrote (including SQL query that builds the
# summary table) as well as data in the RDBMS with the row count (both tables). Don't forget to record the content of the summry table.

# Submit the video and the link to the repository with code to p.lebedev@itransition.com. Make the video publicly accessible.



import json

file_path = 'task1_d.json'

with open(file_path, 'r', encoding='utf-8') as f:
    json_content = f.read()

print(f"output: {json_content[:500]}...")

output: [{:id=>10292064894005717421, :title=>"Look Homeward, Angel", :author=>"Prof. Teressa Kautzer", :genre=>"Humor", :publisher=>"Brill Publishers", :year=>2010, :price=>"$87.25"}, {:id=>13029911509625386835, :title=>"The Yellow Meads of Asphodel", :author=>"Domingo Weimann", :genre=>"Reference book", :publisher=>"Sams Publishing", :year=>2018, :price=>"$31.99"}, {:id=>12880574241579659568, :title=>"A Catskill Eagle", :author=>"Dayle Orn", :genre=>"Comic/Graphic Novel", :publisher=>"Apress", :year=>2...


In [2]:
import re
json_content1 = re.sub(r':([a-zA-Z]+)=>',r'"\1":',json_content)

print(f"output: {json_content1[:500]}...")

output: [{"id":10292064894005717421, "title":"Look Homeward, Angel", "author":"Prof. Teressa Kautzer", "genre":"Humor", "publisher":"Brill Publishers", "year":2010, "price":"$87.25"}, {"id":13029911509625386835, "title":"The Yellow Meads of Asphodel", "author":"Domingo Weimann", "genre":"Reference book", "publisher":"Sams Publishing", "year":2018, "price":"$31.99"}, {"id":12880574241579659568, "title":"A Catskill Eagle", "author":"Dayle Orn", "genre":"Comic/Graphic Novel", "publisher":"Apress", "year":2...


In [3]:
import sqlite3

books_data = json.loads(json_content1)

conn =sqlite3.connect('books.db')
cursor = conn.cursor()

In [4]:
cursor.execute("DROP TABLE IF EXISTS books")

create_table_sql = """
CREATE TABLE IF NOT EXISTS books (
    id TEXT PRIMARY KEY,
    title TEXT,
    author TEXT,
    genre TEXT,
    publisher TEXT,
    year INTEGER,
    price TEXT
)
"""
cursor.execute(create_table_sql)

insert_sql ="""
INSERT INTO books(id, title, author, genre, publisher, year, price )
VALUES (?,?,?,?,?,?,?)
"""

data_insert = [(str(book['id']),book['title'],book['author'],book['genre'],book['publisher'],book['year'],book['price'])for book in books_data]

cursor.executemany(insert_sql,data_insert)
conn.commit()


In [5]:
cursor.execute("DROP TABLE IF EXISTS summary")

summary_sql = """
CREATE TABLE summary AS
SELECT
    year AS publication_year,
    COUNT(*) AS book_count,
    ROUND(AVG(
        CASE
            WHEN price LIKE '€%' THEN 
                CAST(SUBSTR(price, 2) AS REAL) * 1.2 
            ELSE
                CAST(SUBSTR(price, 2) AS REAL)
        END
    ), 2) AS average_price
FROM books
GROUP BY year
ORDER BY year
"""

cursor.execute(summary_sql)
conn.commit()

In [6]:
cursor.execute("SELECT * FROM summary")
for row in cursor.fetchall():
    print(row)

(1871, 43, 48.08)
(1883, 56, 52.51)
(1886, 54, 54.73)
(1904, 37, 54.74)
(1905, 59, 50.62)
(1938, 42, 46.76)
(1955, 49, 54.83)
(1958, 32, 44.17)
(1986, 104, 45.63)
(1987, 120, 50.44)
(1988, 153, 49.91)
(1989, 103, 50.39)
(1990, 122, 52.12)
(1991, 94, 50.01)
(1992, 101, 50.46)
(1993, 114, 51.8)
(1994, 131, 50.55)
(1995, 112, 44.93)
(1996, 111, 54.12)
(1997, 111, 49.94)
(1998, 110, 51.53)
(1999, 108, 52.17)
(2000, 107, 45.64)
(2001, 117, 48.68)
(2002, 101, 49.88)
(2003, 103, 53.53)
(2004, 174, 51.73)
(2005, 116, 51.34)
(2006, 128, 54.89)
(2007, 113, 47.03)
(2008, 92, 50.24)
(2009, 99, 51.01)
(2010, 101, 52.18)
(2011, 122, 51.34)
(2012, 120, 53.64)
(2013, 114, 52.12)
(2014, 100, 53.31)
(2015, 107, 52.77)
(2016, 127, 48.66)
(2017, 119, 52.91)
(2018, 113, 52.28)
(2019, 104, 53.68)
(2020, 107, 51.88)
(2021, 124, 54.1)
(2022, 103, 48.43)
(2023, 94, 49.12)
(2024, 108, 53.81)
(2025, 108, 51.74)
(2026, 116, 50.13)
